This notebook generates the umap embeddings of the images in the dataset. The embeddings are saved in a parquet file.

In [1]:
import argparse
import pathlib

import numpy as np
import pandas as pd
import umap

In [ ]:
# set the arg parser
parser = argparse.ArgumentParser(description="UMAP on a matrix")

parser.add_argument("--data_mode", type=str, default="CP", help="data mode to use")

# get the args
args = parser.parse_args()

# set data mode to either "CP" or "scDINO" or "combined"
data_mode = args.data_mode

In [2]:
# set the paths to the data
CP_fs_sc_profiles_path = pathlib.Path(
    "../../data/feature_selected_data/run_20230920ChromaLiveTL_24hr4ch_MaxIP_norm_fs.parquet"
).resolve(strict=True)
scDINO_sc_profiles_path = pathlib.Path(
    "../../data/outputdir/apoptosis_timelapse/CLS_features/CLS_features_annotated.parquet"
).resolve(strict=True)
combined_profiles_path = pathlib.Path(
    "../../data/20231017ChromaLive_6hr_4ch_MaxIP_normalized_combined_data_feature_selected.parquet"
).resolve(strict=True)

output_path = pathlib.Path(f"../../data/umap/").resolve()
output_path.mkdir(parents=True, exist_ok=True)

In [3]:
if data_mode == "CP":
    # read the data
    profiles_df = pd.read_parquet(CP_fs_sc_profiles_path)
elif data_mode == "scDINO":
    # read the data
    profiles_df = pd.read_parquet(scDINO_sc_profiles_path)
elif data_mode == "combined":
    # read the data
    profiles_df = pd.read_parquet(combined_profiles_path)
else:
    raise ValueError("data_mode must be either 'CP' or 'scDINO' or 'combined'")
print(profiles_df.shape)
# show all columns
pd.set_option("display.max_columns", None)
profiles_df.head()

(236126, 2472)


Metadata_Well Metadata_FOV  Metadata_Time Metadata_ImageNumber   
0          C-02         0001             10                   10  \
1          C-02         0001             10                   10   
2          C-02         0001             10                   10   
3          C-02         0001             10                   10   
4          C-02         0001             10                   10   

  Metadata_Nuclei_Number_Object_Number  Metadata_plate   
0                                   10               1  \
1                                  100               1   
2                                  101               1   
3                                  102               1   
4                                  103               1   

   Metadata_number_of_singlecells Metadata_compound  Metadata_dose   
0                            7914     Staurosporine            0.0  \
1                            7914     Staurosporine            0.0   
2                            7914     Staurosporine            0.0   
3                            7914     Staurosporine            0.0   
4                            7914     Staurosporine            0.0   

  Metadata_control  Metadata_Cells_Number_Object_Number   
0         negative                                    4  \
1         negative                                   93   
2         negative                                   94   
3         negative                                   95   
4         negative                                   96   

   Metadata_Cytoplasm_Parent_Cells  Metadata_Cytoplasm_Parent_Nuclei   
0                                4                                10  \
1                               93                               100   
2                               94                               101   
3                               95                               102   
4                               96                               103   

                  Metadata_Image_FileName_488_1   
0  C-02_F0001_T0010_Z0001_C02_illumcorrect.tiff  \
1  C-02_F0001_T0010_Z0001_C02_illumcorrect.tiff   
2  C-02_F0001_T0010_Z0001_C02_illumcorrect.tiff   
3  C-02_F0001_T0010_Z0001_C02_illumcorrect.tiff   
4  C-02_F0001_T0010_Z0001_C02_illumcorrect.tiff   

                  Metadata_Image_FileName_488_2   
0  C-02_F0001_T0010_Z0001_C03_illumcorrect.tiff  \
1  C-02_F0001_T0010_Z0001_C03_illumcorrect.tiff   
2  C-02_F0001_T0010_Z0001_C03_illumcorrect.tiff   
3  C-02_F0001_T0010_Z0001_C03_illumcorrect.tiff   
4  C-02_F0001_T0010_Z0001_C03_illumcorrect.tiff   

                    Metadata_Image_FileName_561   
0  C-02_F0001_T0010_Z0001_C04_illumcorrect.tiff  \
1  C-02_F0001_T0010_Z0001_C04_illumcorrect.tiff   
2  C-02_F0001_T0010_Z0001_C04_illumcorrect.tiff   
3  C-02_F0001_T0010_Z0001_C04_illumcorrect.tiff   
4  C-02_F0001_T0010_Z0001_C04_illumcorrect.tiff   

                    Metadata_Image_FileName_DNA   
0  C-02_F0001_T0010_Z0001_C01_illumcorrect.tiff  \
1  C-02_F0001_T0010_Z0001_C01_illumcorrect.tiff   
2  C-02_F0001_T0010_Z0001_C01_illumcorrect.tiff   
3  C-02_F0001_T0010_Z0001_C01_illumcorrect.tiff   
4  C-02_F0001_T0010_Z0001_C01_illumcorrect.tiff   

                       Metadata_Image_PathName_488_2   
0  /home/lippincm/Documents/4TB/data/live_cell_ti...  \
1  /home/lippincm/Documents/4TB/data/live_cell_ti...   
2  /home/lippincm/Documents/4TB/data/live_cell_ti...   
3  /home/lippincm/Documents/4TB/data/live_cell_ti...   
4  /home/lippincm/Documents/4TB/data/live_cell_ti...   

                         Metadata_Image_PathName_561   
0  /home/lippincm/Documents/4TB/data/live_cell_ti...  \
1  /home/lippincm/Documents/4TB/data/live_cell_ti...   
2  /home/lippincm/Documents/4TB/data/live_cell_ti...   
3  /home/lippincm/Documents/4TB/data/live_cell_ti...   
4  /home/lippincm/Documents/4TB/data/live_cell_ti...   

                         Metadata_Image_PathName_DNA   
0  /home/lippincm/Documents/4TB/data/live_cell_ti...  \
1  /home/lippincm/Doc

In [4]:
# filter the data and drop nan values
print(profiles_df.shape)
# drop nan values in non metadata columns
profiles_df = profiles_df.dropna(
    subset=profiles_df.columns[~profiles_df.columns.str.contains("Meta")]
)
print(profiles_df.shape)

(236126, 2472)
(236121, 2472)


In [5]:
# get the metadata columns
metadata_cols = profiles_df.columns.str.contains("Metadata_")
metadata_df = profiles_df.loc[:, metadata_cols]
features_df = profiles_df.loc[:, ~metadata_cols]

# set the umap parameters
umap = umap.UMAP(
    n_neighbors=15,
    n_components=2,
    metric="euclidean",
    random_state=42,
    min_dist=0.1,
    n_epochs=500,
    learning_rate=1,
)

# fit the umap model
umap.fit(features_df)

# transform the data
umap_transformed = umap.transform(features_df)

# create a dataframe with the transformed data
umap_df = pd.DataFrame(
    umap_transformed, columns=["UMAP0", "UMAP1"], index=features_df.index
)

# combine the metadata and umap dataframes
umap_df = pd.concat([metadata_df, umap_df], axis=1)
print(umap_df.shape)
umap_df.head()

/home/lippincm/miniforge3/envs/timelapse_env/lib/python3.11/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


(236121, 38)


,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_ImageNumber,Metadata_Nuclei_Number_Object_Number,Metadata_plate,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Image_FileName_488_1,Metadata_Image_FileName_488_2,Metadata_Image_FileName_561,Metadata_Image_FileName_DNA,Metadata_Image_PathName_488_2,Metadata_Image_PathName_561,Metadata_Image_PathName_DNA,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Nuclei_TrackObjects_Displacement_50,Metadata_Nuclei_TrackObjects_DistanceTraveled_50,Metadata_Nuclei_TrackObjects_FinalAge_50,Metadata_Nuclei_TrackObjects_IntegratedDistance_50,Metadata_Nuclei_TrackObjects_Label_50,Metadata_Nuclei_TrackObjects_Lifetime_50,Metadata_Nuclei_TrackObjects_Linearity_50,Metadata_Nuclei_TrackObjects_ParentImageNumber_50,Metadata_Nuclei_TrackObjects_ParentObjectNumber_50,Metadata_Nuclei_TrackObjects_TrajectoryX_50,Metadata_Nuclei_TrackObjects_TrajectoryY_50,Metadata_image_path,Metadata_Nuclei_AreaShape_BoundingBoxArea,Metadata_Nuclei_AreaShape_BoundingBoxMinimum_X,UMAP0,UMAP1
0,C-02,0001,10,10,10,1,7914,Staurosporine,0.0,negative,4,4,10,C-02_F0001_T0010_Z0001_C02_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C03_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C04_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C01_illumcorrect.tiff,/home/lippincm/Documents/4TB/data/live_cell_ti...,/home/lippincm/Documents/4TB/data/live_cell_ti...,/home/lippincm/Documents/4TB/data/live_cell_ti...,381.343487,100.741597,168.002976,7.000000,NaN,246.798653,8,10,0.680729,9,8,7,0,../../data/processed_images/crops/C-02/image_n...,-0.137143,-1.105136,2.939120,-6.382319
1,C-02,0001,10,10,100,1,7914,Staurosporine,0.0,negative,93,93,100,C-02_F0001_T0010_Z0001_C02_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C03_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C04_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C01_illumcorrect.tiff,/home/lippincm/Documents/4TB/data/live_cell_ti...,/home/lippincm/Documents/4TB/data/live_cell_ti...,/home/lippincm/Documents/4TB/data/live_cell_ti...,840.826744,1076.858285,49.648766,5.000000,NaN,72.265563,215,9,0.687032,9,104,-4,-3,../../data/processed_images/crops/C-02/image_n...,-0.697479,-0.211154,1.803937,-3.753607
2,C-02,0001,10,10,101,1,7914,Staurosporine,0.0,negative,94,94,101,C-02_F0001_T0010_Z0001_C02_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C03_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C04_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C01_illumcorrect.tiff,/home/lippincm/Documents/4TB/data/live_cell_ti...,/home/lippincm/Documents/4TB/data/live_cell_ti...,/home/lippincm/Documents/4TB/data/live_cell_ti...,550.188845,1084.777746,81.608823,20.248457,NaN,115.942631,212,9,0.703872,9,102,-7,19,../../data/processed_images/crops/C-02/image_n...,0.840508,-0.787104,1.425470,-4.425173
3,C-02,0001,10,10,102,1,7914,Staurosporine,0.0,negative,95,95,102,C-02_F0001_T0010_Z0001_C02_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C03_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C04_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C01_illumcorrect.tiff,/home/lippincm/Documents/4TB/data/live_cell_ti...,/home/lippincm/Documents/4TB/data/live_cell_ti...,/home/lippincm/Documents/4TB/data/live_cell_ti...,1470.290719,1076.319971,36.013886,20.024984,NaN,113.221331,214,9,0.318084,9,103,1,20,../../data/processed_images/crops/C-02/image_n...,-0.175144,1.024132,3.427536,-3.483689
4,C-02,0001,10,10,103,1,7914,Staurosporine,0.0,negative,96,96,103,C-02_F0001_T0010_Z0001_C02_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C03_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C04_illumcorrect.tiff,C-02_F0001_T0010_Z0001_C01_illumcorrect.tiff,/home/lippincm/Documents/4TB/data/live_cell_ti...,/home/lippincm/Documents/4TB/data/live_cell_ti...,/home/lippincm/Documents/4TB/data/live_cell_ti...,1514.343033,1097.317015,95.634722,11.180340,NaN,146.750055,217,9,0.651684,9,105,5,10,../../data/processed_images/crops/C-02/image_n...,-0.117107,1.095883,4

In [ ]:
# save the umap dataframe
umap_df.to_parquet(f"../../data/umap/{data_mode}_umap_transformed.parquet")